In [1]:
import requests 
import pandas as pd
import json

# I used this API to collect currency exchange rates. like RapidAPI. it has few amount of calls. please try to avoide run this line if possible.  :)))

In [2]:
url = "https://api.apilayer.com/exchangerates_data/timeseries?start_date=2021-10-01&end_date=2021-12-01"

payload = {}
headers= {
  "apikey": "pH2YvFO8Hmc4n9HgLDT28FIEeo4TYSjB"
}

response = requests.request("GET", url, headers=headers, data = payload)

status_code = response.status_code
result = response.text

In [3]:
result = response.json()
result.keys()

dict_keys(['success', 'timeseries', 'start_date', 'end_date', 'base', 'rates'])

# played around and broken down the dictionary into single dict.

In [17]:
first_key = list(result)[0]
first_value = list(result.values())[0]
first_item = list(result.items())[0]
_2nd_item = list(result.items())[1]
_3rd_item = list(result.items())[2]
_4th_item = list(result.items())[3]
_5th_item = list(result.items())[4]
_6th_item = list(result.items())[5]
print(first_key)
print(first_value)
print(first_item)
print(_2nd_item)
print(_3rd_item)
print(_4th_item )
print(_5th_item)
print(_6th_item)
print(type(result['rates']['date'][0]))




success
True
('success', True)
('timeseries', True)
('start_date', '2021-10-01')
('end_date', '2021-12-01')
('base', 'EUR')
('rates', {'2021-10-01': {'AED': 4.259261, 'AFN': 103.784283, 'ALL': 121.927067, 'AMD': 562.752891, 'ANG': 2.080902, 'AOA': 694.570404, 'ARS': 114.456461, 'AUD': 1.597719, 'AWG': 2.087189, 'AZN': 1.975837, 'BAM': 1.957593, 'BBD': 2.340707, 'BDT': 99.305102, 'BGN': 1.955795, 'BHD': 0.437188, 'BIF': 2314.460225, 'BMD': 1.159549, 'BND': 1.574363, 'BOB': 8.004933, 'BRL': 6.222262, 'BSD': 1.159294, 'BTC': 2.4086812e-05, 'BTN': 85.95474, 'BWP': 13.107151, 'BYN': 2.914525, 'BYR': 22727.164537, 'BZD': 2.336824, 'CAD': 1.466807, 'CDF': 2321.41794, 'CHF': 1.078999, 'CLF': 0.033756, 'CLP': 931.419958, 'CNY': 7.475271, 'COP': 4394.69151, 'CRC': 725.728507, 'CUC': 1.159549, 'CUP': 30.728054, 'CVE': 110.679429, 'CZK': 25.320849, 'DJF': 206.075542, 'DKK': 7.43794, 'DOP': 65.457009, 'DZD': 159.26454, 'EGP': 18.228516, 'ERN': 17.398983, 'ETB': 53.575668, 'EUR': 1, 'FJD': 2.458829,

KeyError: 'date'

# another way to break down the keys and values into details.

In [ ]:
for key, value in result.items():
    print(key, value)

In [ ]:
result['rates'] # so we found the realy exchange information is in this dictionary.

## This is the result we need. we will use this result to concatinate into order.csv(assume the data in order.csv is in GBP. so we will use the API collected exchange rate to covert currency in order.csv into euro. )

In [76]:
df=pd.DataFrame(result['rates'])
df_GBP=df.T
df_GBP =pd.DataFrame(df_GBP.GBP).reset_index()

df_GBP.columns=['exchange_rate_date','GBP']
df_GBP.to_csv('EURO2GBP_ex_changerate.csv',index=False)
df_GBP


,exchange_rate_date,GBP
0,2021-10-01,0.856009
1,2021-10-02,0.856072
2,2021-10-03,0.855416
3,2021-10-04,0.853743
4,2021-10-05,0.850910
...,...,...
57,2021-11-27,0.848309
58,2021-11-28,0.846444
59,2021-11-29,0.847970
60,2021-11-30,0.852185


# My first solution is to think about using for loop to write into a total new DF out side of the dictionalry. but you will see they are in a nested dict. that's not useful.

In [ ]:
i= 0
for key, value in result['rates'].items():
    currency_exchange_result = {}
    currency_exchange_result['index']= i
    currency_exchange_result['EUR_GBP_ex_date']=[]
    currency_exchange_result['EUR_GBP_ex_rate']=[]
    currency_exchange_result['EUR_GBP_ex_date'].append(key)
    currency_exchange_result['EUR_GBP_ex_rate'].append(value['GBP'])
    i = i+1
    #print(pd.DataFrame(currency_exchange_result))
    print(currency_exchange_result)



# so I tried to think about some logic. below are the codes I could think about however, it loops infinitive. pls, don't run if you don't want your computer to die.  : ) 


In [ ]:
# i= 0
# for key, value in result['rates'].items():
#     currency_exchange_result = {}
#     currency_exchange_result['EUR_GBP_ex_date']=[]
#     currency_exchange_result['EUR_GBP_ex_rate']=[]
#     while i<len(result['rates']):
#         currency_exchange_result['index']= i
#         i = i+1
#         while i<len(result['rates']):
#             currency_exchange_result['EUR_GBP_ex_date'].append(key)
#             while i<len(result['rates']):
#                 currency_exchange_result['EUR_GBP_ex_rate'].append(value['GBP'])
#     #i = i+1
#     #print(pd.DataFrame(currency_exchange_result))
#     print(currency_exchange_result)


In [ ]:
# i= 0
# for key, value in result['rates'].items():
#     currency_exchange_result = {}
#     currency_exchange_result['EUR_GBP_ex_date']=[]
#     currency_exchange_result['EUR_GBP_ex_rate']=[]
#     while i<3:
#         currency_exchange_result['index']= i
#         i = i+1
#     # while i<3:
#     #     currency_exchange_result['EUR_GBP_ex_date'].append(key)
#     # while i<3:
#     #     currency_exchange_result['EUR_GBP_ex_rate'].append(value['GBP'])
#     #i = i+1
#     #print(pd.DataFrame(currency_exchange_result))
#     print(currency_exchange_result)
